<a href="https://colab.research.google.com/github/CameronDHarris/grrmBot/blob/main/grrmBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import state_ops
from tensorflow.python.framework import ops
from tensorflow.python.training import optimizer

In [3]:
import numpy as np 
from tensorflow import keras
from matplotlib import pyplot as plt
from IPython.display import clear_output

class PlotLearning(keras.callbacks.Callback):
    """
    Callback to plot the learning curves of the model during training.
    """
    def on_train_begin(self, logs={}):
        self.metrics = {}
        for metric in logs:
            self.metrics[metric] = []
            

    def on_epoch_end(self, epoch, logs={}):
        # Storing metrics
        for metric in logs:
            if metric in self.metrics:
                self.metrics[metric].append(logs.get(metric))
            else:
                self.metrics[metric] = [logs.get(metric)]
        
        # Plotting
        metrics = [x for x in logs if 'val' not in x]
        
        f, axs = plt.subplots(1, len(metrics), figsize=(15,5))
        clear_output(wait=True)

        for i, metric in enumerate(metrics):
            axs[i].plot(range(1, epoch + 2), 
                        self.metrics[metric], 
                        label=metric)
            if logs['val_' + metric]:
                axs[i].plot(range(1, epoch + 2), 
                            self.metrics['val_' + metric], 
                            label='val_' + metric)
                
            axs[i].legend()
            axs[i].grid()

        plt.tight_layout()
        plt.show()

In [4]:
class pso(tf.keras.optimizers.Optimizer):
    def __init__(
        self,
        fitness_fn,
        pop_size=100,
        dim=2,
        n_iter=200,
        b=0.9,
        c1=0.8,
        c2=0.5,
        x_min=-1,
        x_max=1,
        name='particleSwarm'
        ):
        self.fitness_fn = fitness_fn
        self.pop_size = pop_size
        self.dim = dim
        self.n_iter = n_iter
        self.b = b
        self.c1 = c1
        self.c2 = c2
        self.x_min = x_min
        self.x_max = x_max
        self.x = self.build_swarm()
        self.p = self.x
        self.f_p = self.fitness_fn(self.x)
        self.fit_history = []
        self.g = self.p[tf.math.argmin(input=self.f_p).numpy()[0]]
        self.v = self.start_velocities()


    def build_swarm(self):
        """Creates the swarm following the selected initialization method. 
        Returns:
            tf.Tensor: The PSO swarm population. Each particle represents a neural
            network. 
        """
        return tf.Variable(
            tf.random.uniform([self.pop_size, self.dim], self.x_min, self.x_max)
        )


    def start_velocities(self):
        """Start the velocities of each particle in the population (swarm). 
        Returns:
            tf.Tensor: The starting velocities.  
        """
        return tf.Variable(
            tf.random.uniform(
                [self.pop_size, self.dim],
                 self.x_min,
                self.x_max ,
            )
        )


    def get_randoms(self):
        """Generate random values to update the particles' positions. 
        Returns:
            _type_: _description_
        """
        return np.random.uniform(0, 1, [2, self.dim])[:, None]

    def update_p_best(self):
        """Updates the *p-best* positions. 
        """
        f_x = self.fitness_fn(self.x)
        self.fit_history.append(tf.reduce_mean(f_x).numpy())
        self.p = tf.where(f_x < self.f_p, self.x, self.p)
        self.f_p = tf.where(f_x < self.f_p, f_x, self.f_p)

    def update_g_best(self):
        """Update the *g-best* position. 
        """
        self.g = self.p[tf.math.argmin(input=self.f_p).numpy()[0]]

    def step(self):
        """It runs ONE step on the particle swarm optimization. 
        """
        r1, r2 = self.get_randoms()
        self.v = (
            self.b * self.v
            + self.c1 * r1 * (self.p - self.x)
            + self.c2 * r2 * (self.g - self.x)
        )
        self.x = tf.clip_by_value(self.x + self.v, self.x_min, self.x_max)
        self.update_p_best()
        self.update_g_best()

    def train(self):
        """The particle swarm optimization. 
        """
        for i in range(self.n_iter):
            self.step()

def objective_function(X):
    return tf.math.sqrt(X[:,0]**2 + X[:,1]**2)[:,None]

def fitness_function():
    def f(X):
        return objective_function(X)
    return f

opt = pso(fitness_fn=fitness_function(), n_iter=100)
opt.train()

# Data Pre-Processing

In [5]:
file_path = 'book1.txt'
text = open(file_path, 'r').read()
print('Length of text in characters: ', len(text))
print('Length of text in tokens: ', len(text.split()))
print('Ratio of characters to tokens:', len(text)/len(text.split()))

Length of text in characters:  370380
Length of text in tokens:  68081
Ratio of characters to tokens: 5.440284367150894


In [16]:
tokens = text.split()
word_lengths = [len(x) for x in tokens]
print(sum(word_lengths) / len(word_lengths))
print(len(set(tokens)))

4.390681687989307
10612


In [6]:
print((len(text) / 100))
print((len(text) / 150))
print((len(text) / 200))

3703.8
2469.2
1851.9


In [18]:
bran_mentions = 0
tyrion_mentions = 0
jamie_mentions = 0
cersei_mentions = 0
tywin_mentions = 0
eddard_mentions = 0
robb_mentions = 0
sansa_mentions = 0
arya_mentions = 0
catelyn_mentions = 0
daenerys_mentions = 0
for token in text.split():
  if token == 'Bran':
    bran_mentions += 1
  elif token == 'Tyrion':
    tyrion_mentions += 1
  elif token == 'Jamie':
    jamie_mentions += 1
  elif token == 'Cersei':
    cersei_mentions += 1
  elif token == 'Tywin':
    tywin_mentions += 1
  elif token == 'Eddard':
    eddard_mentions += 1
  elif token == 'Robb':
    robb_mentions += 1
  elif token == 'Sansa':
    sansa_mentions += 1
  elif token == 'Arya':
    arya_mentions += 1
  elif token == 'Catelyn':
    catelyn_mentions += 1
  elif token == 'Daenerys':
    daenerys_mentions += 1
print(bran_mentions)
print(tyrion_mentions)
print(jamie_mentions)
print(cersei_mentions)
print(tywin_mentions)
print(eddard_mentions)
print(robb_mentions)
print(sansa_mentions)
print(arya_mentions)
print(catelyn_mentions)
print(daenerys_mentions)

153
108
0
17
8
37
79
114
128
141
16


In [ ]:
viewpoint_characters = ['Jamie',
                        'Tyrion',
                        'Cersei',
                        'Jon',
                        'Sansa',
                        'Arya',
                        'Eddard',
                        'Catelyn',
                        'Bran',
                        'Daenerys']

side_characters = ['Tywin',
                   'Kevan',
                   'Gregor',
                   'Sandor',
                   'Petyr',
                   'Varys',
                   'Robb',
                   'Rickson',
                   'Barristan',
                   'Robert',
                   'Aemon',
                   'Samwell',
                   'Jorah',
                   'Viserys',
                   'Drogo',
                   '']
locations = ['Winterfell',
             'King Landing',
             'Casterly Rock',
             'The Wall',
             'The North',
             'Moat Cailin',
             'The Trident',
             'The Eyrie']

In [ ]:
dialogue_strings = text.split('"')
dialogue_strings = dialogue_strings[1::2]

In [ ]:
print('Percentage of dialogue (in characters):', len(' '.join(dialogue_strings)) / len(text))

Percentage of dialogue (in characters): 0.40051028673254496


In [ ]:
vocab = chars = list(set(text))
char_to_ind = {char:i for i, char in enumerate(vocab)}
ind_to_char = np.array(vocab)
encoded_text = np.array([char_to_ind[c] for c in text])

In [ ]:
print(vocab)
print(len(vocab))

['g', '!', 'r', ';', "'", '?', 'h', 'e', '*', ' ', 'p', 'x', ',', 'q', '.', 'f', '\n', 'm', ':', 'a', 's', 'j', 't', 'z', 'c', 'y', 'b', 'd', 'v', 'l', 'w', '"', 'u', 'k', '-', 'n', 'o', 'i']
38


In [ ]:
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

for i in char_dataset.take(500):
     print(ind_to_char[i.numpy()])

b
r
a
n






t
h
e
 
m
o
r
n
i
n
g
 
h
a
d
 
d
a
w
n
e
d
 
c
l
e
a
r
 
a
n
d
 
c
o
l
d
,
 
w
i
t
h
 
a
 
c
r
i
s
p
n
e
s
s
 
t
h
a
t
 
h
i
n
t
e
d
 
a
t
 
t
h
e
 
e
n
d
 
o
f
 
s
u
m
m
e
r
.
 
t
h
e
y
 
s
e
t
 
f
o
r
t
h
 
a
t
 
d
a
y
b
r
e
a
k
 
t
o
 
s
e
e
 
a
 
m
a
n
 
b
e
h
e
a
d
e
d
,
 
t
w
e
n
t
y
 
i
n
 
a
l
l
,
 
a
n
d
 
b
r
a
n
 
r
o
d
e
 
a
m
o
n
g
 
t
h
e
m
,
 
n
e
r
v
o
u
s
 
w
i
t
h
 
e
x
c
i
t
e
m
e
n
t
.
 
t
h
i
s
 
w
a
s
 
t
h
e
 
f
i
r
s
t
 
t
i
m
e
 
h
e
 
h
a
d
 
b
e
e
n
 
d
e
e
m
e
d
 
o
l
d
 
e
n
o
u
g
h
 
t
o
 
g
o
 
w
i
t
h
 
h
i
s
 
l
o
r
d
 
f
a
t
h
e
r
 
a
n
d
 
h
i
s
 
b
r
o
t
h
e
r
s
 
t
o
 
s
e
e
 
t
h
e
 
k
i
n
g
'
s
 
j
u
s
t
i
c
e
 
d
o
n
e
.
 
i
t
 
w
a
s
 
t
h
e
 
n
i
n
t
h
 
y
e
a
r
 
o
f
 
s
u
m
m
e
r
,
 
a
n
d
 
t
h
e
 
s
e
v
e
n
t
h
 
o
f
 
b
r
a
n
'
s
 
l
i
f
e
.






t
h
e
 
m
a
n
 
h
a
d
 
b
e
e
n
 
t
a
k
e
n
 
o
u
t
s
i
d
e
 
a
 
s
m
a
l
l
 
h
o
l
d
f
a
s
t
 
i
n
 
t
h
e
 
h
i
l
l
s
.
 
r
o
b
b
 
t
h
o
u
g
h
t
 
h
e
 
w
a
s
 
a
 
w
i
l
d
l
i


In [ ]:
print(len(text))

370380


In [ ]:
seq_len = 100
sequences = char_dataset.batch(seq_len+1, drop_remainder=True)

In [ ]:
print(sequences)

<BatchDataset element_spec=TensorSpec(shape=(101,), dtype=tf.int64, name=None)>


In [ ]:
def create_seq_targets(seq):
    input_txt = seq[:-1]
    target_txt = seq[1:]
    return input_txt, target_txt
    
dataset = sequences.map(create_seq_targets)

for input_txt, target_txt in  dataset.take(1):
    print(input_txt.numpy())
    print(''.join(ind_to_char[input_txt.numpy()]))
    print('\n')
    print(target_txt.numpy())
    # There is an extra whitespace!
    print(''.join(ind_to_char[target_txt.numpy()]))

[26  2 19 35 16 16 16 22  6  7  9 17 36  2 35 37 35  0  9  6 19 27  9 27
 19 30 35  7 27  9 24 29  7 19  2  9 19 35 27  9 24 36 29 27 12  9 30 37
 22  6  9 19  9 24  2 37 20 10 35  7 20 20  9 22  6 19 22  9  6 37 35 22
  7 27  9 19 22  9 22  6  7  9  7 35 27  9 36 15  9 20 32 17 17  7  2 14
  9 22  6  7]
bran


the morning had dawned clear and cold, with a crispness that hinted at the end of summer. the


[ 2 19 35 16 16 16 22  6  7  9 17 36  2 35 37 35  0  9  6 19 27  9 27 19
 30 35  7 27  9 24 29  7 19  2  9 19 35 27  9 24 36 29 27 12  9 30 37 22
  6  9 19  9 24  2 37 20 10 35  7 20 20  9 22  6 19 22  9  6 37 35 22  7
 27  9 19 22  9 22  6  7  9  7 35 27  9 36 15  9 20 32 17 17  7  2 14  9
 22  6  7 25]
ran


the morning had dawned clear and cold, with a crispness that hinted at the end of summer. they


In [ ]:
batch_size = 128
buffer_size = 10000
dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

## EDA

In [ ]:
all_freq = {}

for i in test_str:
    if i in all_freq:
        all_freq[i] += 1
    else:
        all_freq[i] = 1

SyntaxError: ignored

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# split text into windows of 200 characters
window_length = 500
text_windows = []
for i in range(0, len(text), window_length):
  text_in_window = text[i:i+window_length]
  text_windows.append(text_in_window)



In [ ]:
print(text_windows)

["bran\n\n\nthe morning had dawned clear and cold, with a crispness that hinted at the end of summer. they set forth at daybreak to see a man beheaded, twenty in all, and bran rode among them, nervous with excitement. this was the first time he had been deemed old enough to go with his lord father and his brothers to see the king's justice done. it was the ninth year of summer, and the seventh of bran's life.\n\n\nthe man had been taken outside a small holdfast in the hills. robb thought he was a wildli", "ng, his sword sworn to mance rayder, the king-beyond-the-wall. it made bran's skin prickle to think of it. he remembered the hearth tales old nan told them. the wildlings were cruel men, she said, slavers and slayers and thieves. they consorted with giants and ghouls, stole girl children in the dead of night, and drank blood from polished horns. and their women lay with the others in the long night to sire terrible half-human children.\n\n\nbut the man they found bound hand and foot 

# Model Training

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Dec 22 21:16:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P0    29W /  70W |  14390MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 16.0 MB/s 


## SimpleRNN Model

In [ ]:
import tensorflow_addons as tfa
#from tfa.losses import SigmoidFocalCrossEntropy
from tensorflow.keras.layers import RNN
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.losses import sparse_categorical_crossentropy

def sparse_cat_loss(y_true,y_pred):
  return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embed_dim,batch_input_shape=[batch_size, None]))
    model.add(LSTM(rnn_neurons,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
    # Final Dense Layer to Predict
    model.add(Dense(vocab_size))
    model.compile(optimizer='Adam', loss=sparse_cat_loss) 
    return model
  
  
rnn_neurons = 1024
 # Length of the vocabulary in chars
vocab_size = len(vocab)
# The embedding dimension
embed_dim = 64
# Number of RNN units

model = None
#Create the model
model = create_model(
  vocab_size = vocab_size,
  embed_dim=embed_dim,
  rnn_neurons=rnn_neurons,
  batch_size=batch_size)

callback_list = [PlotLearning()]
#Train the model
epochs = 200
model.fit(dataset,epochs=epochs)

Epoch 1/200
28/28 [==============================] - 5s 102ms/step - loss: 3.2592
Epoch 2/200
28/28 [==============================] - 3s 102ms/step - loss: 2.9430
Epoch 3/200
28/28 [==============================] - 3s 102ms/step - loss: 2.8365
Epoch 4/200
28/28 [==============================] - 3s 102ms/step - loss: 2.6520
Epoch 5/200
28/28 [==============================] - 3s 104ms/step - loss: 2.4550
Epoch 6/200
28/28 [==============================] - 3s 103ms/step - loss: 2.3342
Epoch 7/200
28/28 [==============================] - 3s 104ms/step - loss: 2.2622
Epoch 8/200
28/28 [==============================] - 3s 104ms/step - loss: 2.2037
Epoch 9/200
28/28 [==============================] - 3s 105ms/step - loss: 2.1521
Epoch 10/200
28/28 [==============================] - 3s 106ms/step - loss: 2.1019
Epoch 11/200
28/28 [==============================] - 3s 107ms/step - loss: 2.0569
Epoch 12/200
28/28 [==============================] - 3s 107ms/step - loss: 2.0079
Epoch 13/200


In [ ]:
from tensorflow.keras.models import load_model
model.save('grrm_gen_lstm.h5') 

#Currently our model only expects 128 sequences at a time. We can create a new model that only expects a batch_size=1. We can create a new model with this batch size, then load our saved models weights.
#Then call .build() on the mode

model_gen1 = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1)
model_gen1.load_weights('grrm_gen_lstm.h5')
model_gen1.build(tf.TensorShape([1, None]))


def generate_text(model, start_seed,gen_size=100,temp=1.0):
  '''
  model: Trained Model to Generate Text
  start_seed: Intial Seed text in string form
  gen_size: Number of characters to generate
  Basic idea behind this function is to take in some seed text, format it so
  that it is in the correct shape for our network, then loop the sequence as
  we keep adding our own predicted characters. Similar to our work in the RNN
  time series problems.
  '''
  # Number of characters to generate
  num_generate = gen_size
  # Vecotrizing starting seed text
  input_eval = [char_to_ind[s] for s in start_seed]
  # Expand to match batch format shape
  input_eval = tf.expand_dims(input_eval, 0)
  # Empty list to hold resulting generated text
  text_generated = []
  # Temperature effects randomness in our resulting text
  # The term is derived from entropy/thermodynamics.
  # The temperature is used to effect probability of next characters.
  # Higher probability == lesss surprising/ more expected
  # Lower temperature == more surprising / less expected
  temperature = temp
  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      # Generate Predictions
      predictions = model(input_eval)
      # Remove the batch shape dimension
      predictions = tf.squeeze(predictions, 0)
      # Use a cateogircal disitribution to select the next character
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      # Pass the predicted charracter for the next input
      input_eval = tf.expand_dims([predicted_id], 0)
      # Transform back to character letter
      text_generated.append(ind_to_char[predicted_id])
  return (start_seed + ''.join(text_generated))


print(generate_text(model_gen1,"bran",gen_size=1000))

brand my repulsive nephew will be king after jon my mother's trueborn son of lannister."


"oh?" littlefinger's replied with a col had her own so much as he reached loudly. he was still shy of thirty. they went let a dagger than the the time jon felt a corn on the free cities, and terrible sundered him. they knew you could even he let the night's watch."


tyrion nodded.


jon snow set his mouth in a grim line. "if that's what it is."


tyrion grew clisk that his gods had atcher will be proud of bran," illyrio said. "he will have the cresuin rall the window to scream for help, but the man more than direckly.


the changions; it dumpy eyes like littlefinger.


at the foot of the steps walled in way, snow us and after the sifter of pertos. will have it back." viserys said, smiling his fuce to take me with you when you go back to the wall," jon said in a sudden rush. "father will give me leave, and her brother did not now be leave them all behind and just keep going."


othe golden hair a

In [ ]:
print(generate_text(model_gen1,text,gen_size=10000))

## Classic LSTM Model

In [ ]:
import tensorflow_addons as tfa
#from tfa.losses import SigmoidFocalCrossEntropy
from tensorflow.keras.layers import RNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.losses import sparse_categorical_crossentropy
rnn_neurons = 512
LSTMCell = tfa.rnn.PeepholeLSTMCell(rnn_neurons)

model_opt = tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=True,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    name='Adam'
)


def sparse_cat_loss(y_true,y_pred):
  return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embed_dim, batch_input_shape=[batch_size, None]))
    model.add(LSTM(1024, return_sequences=True, recurrent_initializer='glorot_uniform', recurrent_activation='swish'))
    #model.add(RNN(LSTMCell, return_sequences=True))
    #model.add(RNN(512, return_sequences=True))
    #model.add(RNN(LSTMCell, return_sequences=True))
    #model.add(LSTM(512, return_sequences=True, recurrent_initializer='glorot_uniform'))
    #model.add(LSTM(256, return_sequences=True, recurrent_initializer='glorot_uniform'))
    #model.add(Dense(128, activation='sigmoid'))
    #model.add(LSTM(1024, return_sequences=True))
    #model.add(RNN(LSTMCell, return_sequences=True))
    #model.add(LSTM(512,return_sequences=True))
    #model.add(Dropout(0.01))
    # Final Dense Layer to Predict
    model.add(Dense(vocab_size, activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy') 
    return model
  
  
  
 # Length of the vocabulary in chars
vocab_size = len(vocab)
# The embedding dimension
embed_dim = 2048
# Number of RNN units

model = None
#Create the model
model = create_model(
  vocab_size = vocab_size,
  embed_dim=embed_dim,
  rnn_neurons=rnn_neurons,
  batch_size=batch_size)

callback_list = [PlotLearning()]
#Train the model
epochs = 1800
model.fit(dataset,epochs=epochs)

Epoch 1/1800


ValueError: ignored

In [ ]:
from tensorflow.keras.models import load_model
#model.save('grrm_gen.h5') 

#Currently our model only expects 128 sequences at a time. We can create a new model that only expects a batch_size=1. We can create a new model with this batch size, then load our saved models weights.
#Then call .build() on the mode

model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1)
model.load_weights('grrm_gen.h5')
model.build(tf.TensorShape([1, None]))


def generate_text(model, start_seed,gen_size=100,temp=1.0):
  '''
  model: Trained Model to Generate Text
  start_seed: Intial Seed text in string form
  gen_size: Number of characters to generate
  Basic idea behind this function is to take in some seed text, format it so
  that it is in the correct shape for our network, then loop the sequence as
  we keep adding our own predicted characters. Similar to our work in the RNN
  time series problems.
  '''
  # Number of characters to generate
  num_generate = gen_size
  # Vecotrizing starting seed text
  input_eval = [char_to_ind[s] for s in start_seed]
  # Expand to match batch format shape
  input_eval = tf.expand_dims(input_eval, 0)
  # Empty list to hold resulting generated text
  text_generated = []
  # Temperature effects randomness in our resulting text
  # The term is derived from entropy/thermodynamics.
  # The temperature is used to effect probability of next characters.
  # Higher probability == lesss surprising/ more expected
  # Lower temperature == more surprising / less expected
  temperature = temp
  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      # Generate Predictions
      predictions = model(input_eval)
      # Remove the batch shape dimension
      predictions = tf.squeeze(predictions, 0)
      # Use a cateogircal disitribution to select the next character
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      # Pass the predicted charracter for the next input
      input_eval = tf.expand_dims([predicted_id], 0)
      # Transform back to character letter
      text_generated.append(ind_to_char[predicted_id])
  return (start_seed + ''.join(text_generated))


print(generate_text(model,"bran",gen_size=1000))

OSError: ignored

In [ ]:
seed = text[0:1000]
print(generate_text(model,seed,gen_size=10000))

## Peephole LSTM Model

In [ ]:
#import tensorflow_addons as tfa
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.losses import sparse_categorical_crossentropy
rnn_neurons = 1024
LSTMCell = tfa.rnn.PeepholeLSTMCell(rnn_neurons, recurrent_initializer='glorot_uniform')

def sparse_cat_loss(y_true,y_pred):
  return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embed_dim, batch_input_shape=[batch_size, None]))
    model.add(RNN(LSTMCell, return_sequences=True, stateful=True))
    #model.add(LSTM(128, return_sequences=True))
    # Final Dense Layer to Predict
    model.add(Dense(vocab_size))
    model.compile(optimizer='Adam', loss=sparse_cat_loss) 
    return model
  
  
  
 # Length of the vocabulary in chars
vocab_size = len(vocab)
# The embedding dimension
embed_dim = 64


model2 = None
#Create the model
model2 = create_model(
  vocab_size = vocab_size,
  embed_dim=embed_dim,
  rnn_neurons=rnn_neurons,
  batch_size=batch_size)


#Train the model
epochs = 200
model2.fit(dataset,epochs=epochs)

Epoch 1/200
28/28 [==============================] - 7s 191ms/step - loss: 3.0735
Epoch 2/200
28/28 [==============================] - 6s 192ms/step - loss: 2.8858
Epoch 3/200
28/28 [==============================] - 6s 192ms/step - loss: 2.5797
Epoch 4/200
28/28 [==============================] - 6s 211ms/step - loss: 2.3527
Epoch 5/200
28/28 [==============================] - 6s 195ms/step - loss: 2.2531
Epoch 6/200
28/28 [==============================] - 6s 195ms/step - loss: 2.1599
Epoch 7/200
28/28 [==============================] - 6s 194ms/step - loss: 2.0774
Epoch 8/200
28/28 [==============================] - 6s 196ms/step - loss: 1.9986
Epoch 9/200
28/28 [==============================] - 6s 194ms/step - loss: 1.9249
Epoch 10/200
28/28 [==============================] - 6s 191ms/step - loss: 1.8540
Epoch 11/200
28/28 [==============================] - 6s 191ms/step - loss: 1.7850
Epoch 12/200
28/28 [==============================] - 6s 190ms/step - loss: 1.7185
Epoch 13/200


In [ ]:
model2.save('grrm_gen_peephole.h5')

model_gen = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1)
model_gen.load_weights('grrm_gen_peephole.h5')
model_gen.build(tf.TensorShape([1, None]))

In [ ]:
print(generate_text(model_gen,"bran",gen_size=1000))

brandon and easi robb, in great would be on a king's blood, awhild t saw the hound standing with young joffrey as squires swarmed around them. "at least thin to deplace for him at court. and spent as a field of new-fallen snow, with silver chasings and clasps that glittered in the sun was like this knew beneath the sun. when he had ones looked for the king. jon was s, this man who passes the sentente some ponsty watchthe ckrtain the d among them, years. my son is first time in his minechat winterfell with his son lingering in the shadow of death."


"his son . . . " ned began.


"he should have thought to ver when tyrion had told him of his intentions. "i warn you, lannister, you'll find no inns and the stars promise, i would stepped forward and put his hands on her waists. he never see bran stronger.


a hot was grievous sad to hear about your troubles on the kingsroad."


that did not feel in with close to dany on her silver, dus until a sweet lord stark of winterfell.


there were e

In [ ]:
seed = text[0:1000]
print(generate_text(model_gen,seed,gen_size=10000))

bran


the morning had dawned clear and cold, with a crispness that hinted at the end of summer. they set forth at daybreak to see a man beheaded, twenty in all, and bran rode among them, nervous with excitement. this was the first time he had been deemed old enough to go with his lord father and his brothers to see the king's justice done. it was the ninth year of summer, and the seventh of bran's life.


the man had been taken outside a small holdfast in the hills. robb thought he was a wildling, his sword sworn to mance rayder, the king-beyond-the-wall. it made bran's skin prickle to think of it. he remembered the hearth tales old nan told them. the wildlings were cruel men, she said, slavers and slayers and thieves. they consorted with giants and ghouls, stole girl children in the dead of night, and drank blood from polished horns. and their women lay with the others in the long night to sire terrible half-human children.


but the man they found bound hand and foot to the holdfast

In [ ]:
print(generate_text(model_gen,text,gen_size=10000))

bran


the morning had dawned clear and cold, with a crispness that hinted at the end of summer. they set forth at daybreak to see a man beheaded, twenty in all, and bran rode among them, nervous with excitement. this was the first time he had been deemed old enough to go with his lord father and his brothers to see the king's justice done. it was the ninth year of summer, and the seventh of bran's life.


the man had been taken outside a small holdfast in the hills. robb thought he was a wildling, his sword sworn to mance rayder, the king-beyond-the-wall. it made bran's skin prickle to think of it. he remembered the hearth tales old nan told them. the wildlings were cruel men, she said, slavers and slayers and thieves. they consorted with giants and ghouls, stole girl children in the dead of night, and drank blood from polished horns. and their women lay with the others in the long night to sire terrible half-human children.


but the man they found bound hand and foot to the holdfast

# LSTM with Focal Loss

In [ ]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

maxlen = 200
step = 20
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

total chars: 38
nb sequences: 18509


<ipython-input-65-1f0b0160da87>:15: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
<ipython-input-65-1f0b0160da87>:16: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(chars)), dtype=np.bool)


In [ ]:
model3 = Sequential()
model3.add(GRU(1024, batch_input_shape=(256, maxlen, len(chars)), stateful=True, recurrent_initializer='glorot_uniform'))
model3.add(Dense(len(chars), activation='softmax'))

model3.compile(loss=tfa.losses.SigmoidFocalCrossEntropy(), optimizer='adam')

In [ ]:
model3.fit(x, y, batch_size=256, epochs=500)

Epoch 1/500
72/73 [============================>.] - ETA: 0s - loss: 0.7271

InvalidArgumentError: ignored

In [ ]:
model3.save('grrm_gen_focalloss.h5')

In [ ]:
def generate_text(model, start_seed,gen_size=100,temp=1.0):
  '''
  model: Trained Model to Generate Text
  start_seed: Intial Seed text in string form
  gen_size: Number of characters to generate
  Basic idea behind this function is to take in some seed text, format it so
  that it is in the correct shape for our network, then loop the sequence as
  we keep adding our own predicted characters. Similar to our work in the RNN
  time series problems.
  '''
  # Number of characters to generate
  num_generate = gen_size
  # Vecotrizing starting seed text
  input_eval = [char_indices[s] for s in start_seed]
  # Expand to match batch format shape
  input_eval = tf.expand_dims(input_eval, 0)
  # Empty list to hold resulting generated text
  text_generated = []
  # Temperature effects randomness in our resulting text
  # The term is derived from entropy/thermodynamics.
  # The temperature is used to effect probability of next characters.
  # Higher probability == lesss surprising/ more expected
  # Lower temperature == more surprising / less expected
  temperature = temp
  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      # Generate Predictions
      predictions = model(input_eval)
      # Remove the batch shape dimension
      predictions = tf.squeeze(predictions, 0)
      # Use a cateogircal disitribution to select the next character
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      # Pass the predicted charracter for the next input
      input_eval = tf.expand_dims([predicted_id], 0)
      # Transform back to character letter
      text_generated.append(indices_char[predicted_id])
  return (start_seed + ''.join(text_generated))

### Text Generation

In [ ]:
import sys

In [ ]:
#saved_model= tf.keras.models.load_model('grrm_gen_focalloss.h5')

In [ ]:
book_beginning = """Bran


The morning had dawned clear and cold, with a crispness that hinted at the end of summer. They set forth at daybreak to see a man beheaded, twenty in all, and Bran rode among them, nervous with excitement. This was the first time he had been deemed old enough to go with his lord father and his brothers to see the king's justice done. It was the ninth year of summer, and the seventh of Bran's life.


The man had been taken outside a small holdfast in the hills. Robb thought he was a wildling, his sword sworn to Mance Rayder, the King-beyond-the-Wall. It made Bran's skin prickle to think of it. He remembered the hearth tales Old Nan told them. The wildlings were cruel men, she said, slavers and slayers and thieves. They consorted with giants and ghouls, stole girl children in the dead of night, and drank blood from polished horns. And their women lay with the Others in the Long Night to sire terrible half-human children."""

In [ ]:
def generate_from_seed(model, seed, gen_size, temps):
    print("******************************************************")
    print('Generating with seed: ', seed)

    for temperature in temps:
        print('----- temperature:', temperature)

        generated = ''
        sentence = seed[0:maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(gen_size):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
        
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
print(generate_from_seed(model3,"bran",gen_size=1000, temps=[1.0, 0.1, 5.0]))

******************************************************
Generating with seed:  bran
----- temperature: 1.0
----- Generating with seed: "bran"
bran nwoetyli,i liiidlwiiai yy.rltencenled i tewnm

KeyboardInterrupt: ignored

In [ ]:
print(generate_from_seed(model3,book_beginning.lower(),gen_size=1000, temps=[1.0, 0.1, 5.0]))

******************************************************
Generating with seed:  bran


the morning had dawned clear and cold, with a crispness that hinted at the end of summer. they set forth at daybreak to see a man beheaded, twenty in all, and bran rode among them, nervous with excitement. this was the first time he had been deemed old enough to go with his lord father and his brothers to see the king's justice done. it was the ninth year of summer, and the seventh of bran's life.


the man had been taken outside a small holdfast in the hills. robb thought he was a wildling, his sword sworn to mance rayder, the king-beyond-the-wall. it made bran's skin prickle to think of it. he remembered the hearth tales old nan told them. the wildlings were cruel men, she said, slavers and slayers and thieves. they consorted with giants and ghouls, stole girl children in the dead of night, and drank blood from polished horns. and their women lay with the others in the long night to sire terrible hal

KeyboardInterrupt: ignored